In [1]:
from __future__ import division
import cv2
from mtcnn.mtcnn import MTCNN
from eye import Eye
from calibration import Calibration
import dlib
import os
import numpy as np
import json
import os

Using TensorFlow backend.


In [2]:
calibration = Calibration()
_predictor = dlib.shape_predictor("trained_models/shape_predictor_68_face_landmarks.dat")

In [ ]:
cam = cv2.VideoCapture(0)
while True:
    ret,frame = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        continue
    if key_pressed == ord('q'):
        break
    frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(frame)
    for i in range(len(faces)) :
        
        #eye gaze --------------------------------------------------------
        
        eye_left = None
        eye_right = None
        bounding_box = faces[i]['box']
        x1, y1, width, height = bounding_box
#         face = [(x1,y1),(x1+width,y1+height)]
        face = dlib.rectangle(x1,y1,x1+width,y1+height)
        landmarks = _predictor(frame, face)
        left_eye = []
        right_eye = []
        left_eye.append([landmarks.part(36).x,landmarks.part(36).y])
        left_eye.append([landmarks.part(37).x,landmarks.part(37).y])
        left_eye.append([landmarks.part(38).x,landmarks.part(38).y])
        left_eye.append([landmarks.part(39).x,landmarks.part(39).y])
        left_eye.append([landmarks.part(40).x,landmarks.part(40).y])
        left_eye.append([landmarks.part(41).x,landmarks.part(41).y])
        
        right_eye.append([landmarks.part(42).x,landmarks.part(42).y])
        right_eye.append([landmarks.part(43).x,landmarks.part(43).y])
        right_eye.append([landmarks.part(44).x,landmarks.part(44).y])
        right_eye.append([landmarks.part(45).x,landmarks.part(45).y])
        right_eye.append([landmarks.part(46).x,landmarks.part(46).y])
        right_eye.append([landmarks.part(47).x,landmarks.part(47).y])
        
        left_eye = np.array(left_eye)
        right_eye = np.array(right_eye)
        
        cv2.polylines(frame, [left_eye],True,(0,255,0),1)
        cv2.polylines(frame, [right_eye],True,(0,255,0),1)
        
        cv2.rectangle(frame,(bounding_box[0], bounding_box[1]),(bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),(0,155,255),2)
  
    cv2.imshow("faces",frame)


cam.release()
cv2.destroyAllWindows()

In [3]:
def process_json_list(json_list,img):
    ldmks = [eval(s) for s in json_list]
    return np.array([(x, img.shape[0]-y, z) for (x,y,z) in ldmks])

In [4]:
def crop_eyes(path,dst_path):
    for root,dirs,files in os.walk(path):
        for q,file in enumerate(files):
            if file.endswith(".json"):
                with open(path+'/'+file) as json_file: 
                    data = json.load(json_file)
                img = cv2.imread(path+"/"+file.split('.')[0]+".jpg")
                
                
                ldmks_interior_margin = process_json_list( data['interior_margin_2d'],img)
                ldmks_interior_margin =  np.array([ldmks_interior_margin[:,:2]], int)[0]
#                 print(ldmks_interior_margin)
                rect = cv2.boundingRect(ldmks_interior_margin)
                x,y,w,h = rect
                croped = img[y:y+h, x:x+w].copy()
                
                ldmks_interior_margin = ldmks_interior_margin - ldmks_interior_margin.min(axis=0)
#                 print(ldmks_interior_margin)
                cv2.polylines(croped, np.array([ldmks_interior_margin], int), True, (0,0,0), 2)
#                 cv2.imshow("croped",croped)
                mask = np.zeros(croped.shape[:2], np.uint8)
                
                cv2.drawContours(mask, [ldmks_interior_margin], -1, (255, 0, 0), -1, cv2.LINE_AA)
#                 cv2.imshow("mask",mask)
                dst = cv2.bitwise_and(croped, croped, mask=mask)
#                 cv2.imshow("sds",dst)
                
                cv2.imwrite(dst_path+"/"+file.split('.')[0]+".jpg",dst)
                if q%200==0:
                    print(q)
        

In [6]:
crop_eyes("./final_dataset/UnityEyes_Windows/extreme","./final_dataset/UnityEyes_Windows/cropped_extreme")

In [ ]:
eye

In [ ]:
eye_array = []
for i in eye:
    i = i[1:-1]
    i = i.split(',')
    eye_array.append([round(float(i[0])),round(float(i[1]))])
eye_array = np.array(eye_array)

In [ ]:
eye_array

In [ ]:
cv2.polylines(eye1, [eye_array],True,(0,255,0),1)
cv2.imshow("eye",eye1)
cv2.waitKey()

In [ ]:
import json
img = cv2.imread("./final_dataset/UnityEyes_Windows/imgs/1.jpg")
with open('./final_dataset/UnityEyes_Windows/imgs/1.json') as json_file: 
    data = json.load(json_file)



In [ ]:
def process_json_list(json_list):
    ldmks = [eval(s) for s in json_list]
    return np.array([(x, img.shape[0]-y, z) for (x,y,z) in ldmks])

In [ ]:
ldmks_interior_margin = process_json_list( data['interior_margin_2d'])

In [ ]:
cv2.polylines(img, np.array([ldmks_interior_margin[:,:2]], int), True, (0,0,0), 2)


In [ ]:
cv2.imshow("eye",img)
cv2.waitKey()